In [2]:
import torch
from torchvision import models, transforms
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog, Label
import ttkbootstrap as ttk
import json
import urllib.request

# Load pre-trained model (ResNet18)
model = models.resnet18(pretrained=True)
model.eval()

# Load ImageNet class labels
LABELS_URL = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
with urllib.request.urlopen(LABELS_URL) as f:
    classes = [line.strip().decode('utf-8') for line in f.readlines()]

# Image preprocessing
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Prediction function
def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(image).unsqueeze(0)

    with torch.no_grad():
        output = model(input_tensor)
        _, predicted = torch.max(output, 1)

    return classes[predicted.item()], image

# UI setup
app = ttk.Window(themename="cosmo")
app.title("Image Classifier (No TensorFlow)")
app.geometry("600x500")

title = ttk.Label(app, text="Image Classifier", font=("Helvetica", 20, 'bold'))
title.pack(pady=10)

img_label = Label(app)
img_label.pack()

result_label = ttk.Label(app, text="", font=("Helvetica", 16))
result_label.pack(pady=20)

def upload_and_classify():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp")])
    if file_path:
        prediction, image = predict_image(file_path)

        # Show image in UI
        image.thumbnail((300, 300))
        img = ImageTk.PhotoImage(image)
        img_label.configure(image=img)
        img_label.image = img

        # Show prediction
        result_label.config(text=f"Prediction: {prediction}")

upload_btn = ttk.Button(app, text="Upload Image", command=upload_and_classify)
upload_btn.pack(pady=10)

app.mainloop()
